### Important Note for question1 !
- Please **do not** change the default variable names in this problem, as we will use them in different parts.
- The default variables are initially set to "None".
- You only need to modify code in the "TODO" part. We added a lot of "assertions" to check your code. **Do not** modify them.

In [1]:
# load packages
import numpy as np
import pandas as pd
import time
from sklearn.naive_bayes import GaussianNB

## P1. Load data and plot
### TODO
- Load train and test data, and split them into inputs(trainX, testX) and labels(trainY, testY)




In [2]:
# Use pandas to load q1_train.csv and q1_test.csv
# Each data point has 200 features(X), followed by 1 label(Y)

#### TODO ####
trainX = pd.read_csv('q1_train.csv').iloc[:,1:201].values
trainY = pd.read_csv('q1_train.csv',).iloc[:,201].values

testX = pd.read_csv('q1_test.csv').iloc[:,1:201].values
testY = pd.read_csv('q1_test.csv').iloc[:,201].values
##############

assert(len(trainX.shape) == 2)
assert(len(trainY.shape) == 1)
assert(trainX.shape[1] == 200)

## P2. Write your Gaussian NB solver
### TODO
- Finish the myNBSolver() function. 
    - Compute P(y == 0) and P(y == 1), saved in "py0" and "py1"
    - Compute mean/variance of trainX for both y = 0 and y = 1, saved in "mean0", "var0", "mean1" and "var1"
        - Each of them should have shape (M), where M is number of features.
    - Compute P(xi | y == 0) and P(xi | y == 1), compare and save **binary** prediction in "train_pred" and "test_pred"
    - Compute train accuracy and test accuracy, saved in "train_acc" and "test_acc".
    - Return train accuracy and test accuracy.

In [3]:
def myNBSolver(trainX, trainY, testX, testY):
    
    N_train = trainX.shape[0]
    N_test = testX.shape[0]
    M = trainX.shape[1]
    
    #### TODO ####
    # Compute P(y == 0) and P(y == 1)
    
    py0 = np.count_nonzero(trainY == 0)/len(trainY)
    py1 = np.count_nonzero(trainY == 1)/len(trainY)
  
    ##############
    print("Total probablity is %.2f. Should be equal to 1." %(py0 + py1))

    #### TODO ####
    # Compute mean/var for each label

    mean0 = (np.sum(trainX[trainY == 0],axis = 0)/np.count_nonzero(trainY == 0)).reshape(1,-1)
    mean1 = (np.sum(trainX[trainY == 1],axis = 0)/np.count_nonzero(trainY == 1)).reshape(1,-1)
    
    var0 = ((np.sum(np.square(trainX[trainY == 0] - mean0),axis = 0))/np.count_nonzero(trainY == 0)).reshape(1,-1)
    var1 = ((np.sum(np.square(trainX[trainY == 1] - mean1),axis = 0))/np.count_nonzero(trainY == 1)).reshape(1,-1)
 
    mean0 = mean0.reshape(-1,1)
    ##############
    assert(mean0.shape[0] == M)

    mean0 = mean0.reshape(1,-1)
    Pxy0_train = (np.exp(-0.5*(np.divide(np.square(trainX - mean0),var0))))/((2*np.pi)**0.5)
    Pxy0_train = np.product(np.divide(Pxy0_train,np.sqrt(var0)),axis=1)
    
    Pxy1_train = (np.exp(-0.5*(np.divide(np.square(trainX - mean1),var1))))/((2*np.pi)**0.5)
    Pxy1_train = np.product(np.divide(Pxy1_train,np.sqrt(var1)),axis=1)

    #### TODO ####
    Pyx0_train = Pxy0_train*py0
    Pyx1_train = Pxy1_train*py1
    
    #print(Pyx0_train.min())
    # Compute P(xi|y == 0) and P(xi|y == 1), compare and make prediction
    # This part may spend 5 - 10 minutes or even more if you use for loop, so feel free to 
    # print something (like step number) to check the progress
    
    train_pred = np.empty((N_train))
    train_pred[np.where(Pyx0_train >= Pyx1_train)] = 0
    train_pred[np.where(Pyx0_train < Pyx1_train)] = 1

    Pxy0_test = (np.exp(-0.5*(np.divide(np.square(testX - mean0),var0))))/((2*np.pi)**0.5)
    Pxy0_test = np.product(np.divide(Pxy0_test,np.sqrt(var0)),axis=1)

    Pxy1_test = (np.exp(-0.5*(np.divide(np.square(testX - mean1),var1))))/((2*np.pi)**0.5)
    Pxy1_test = np.product(np.divide(Pxy1_test,np.sqrt(var1)),axis=1)

    Pyx0_test = Pxy0_test*py0
    Pyx1_test = Pxy1_test*py1

    test_pred = np.empty((N_test))
    test_pred[np.where(Pyx0_test >= Pyx1_test)] = 0
    test_pred[np.where(Pyx0_test < Pyx1_test)] = 1

    ##############
    assert(train_pred[0] == 0 or train_pred[0] == 1)
    assert(test_pred[0] == 0 or test_pred[0] == 1)
    
    #### TODO ####
    # Compute train accuracy and test accuracy
    
    train_acc = np.count_nonzero(train_pred == trainY)/N_train
    test_acc = np.count_nonzero(test_pred == testY)/N_test
    
    ##############
    
    return train_acc, test_acc

In [4]:
# driver to test your NB solver
train_acc, test_acc = myNBSolver(trainX, trainY, testX, testY)
print("Train accuracy is %.2f" %(train_acc * 100))
print("Test accuracy is %.2f" %(test_acc * 100))

Total probablity is 1.00. Should be equal to 1.
Train accuracy is 92.22
Test accuracy is 92.05


## P3. Test your result using sklearn
### TODO
- Finish the skNBSolver() function. 
     - fit model, make prediction and return accuracy for train and test sets.

In [5]:
def skNBSolver(trainX, trainY, testX, testY):
    
    #### TODO ####
    # fit model
    # make prediction
    # compute accuracy
    gnb = GaussianNB()
    gnb.fit(trainX, trainY)
    y_pred_train = gnb.predict(trainX) 
    y_pred_test = gnb.predict(testX)
    acc_train = (y_pred_train != trainY).sum() 
    acc_test = (y_pred_test != testY).sum()
    sk_train_acc = 1- (acc_train/trainX.shape[0])
    sk_test_acc = 1 - (acc_test/testX.shape[0])
    
    ##############
    return sk_train_acc, sk_test_acc
    

In [6]:
# driver to test skNBSolver
sk_train_acc, sk_test_acc = skNBSolver(trainX, trainY, testX, testY)
print("Train accuracy is %.2f" %(sk_train_acc * 100))
print("Test accuracy is %.2f" %(sk_test_acc * 100))

Train accuracy is 92.22
Test accuracy is 92.05
